# dependencies and imports

In [1]:
!pip install mediapipe opencv-python

In [2]:
!pip install tensorflow
!pip install keras

In [2]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
import tensorflow as tf
import pandas as pd
from tensorflow import keras

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_holistic = mp.solutions.holistic

# get model and predict function

In [4]:
model = keras.models.load_model('ASL_Model_3.h5',compile = True)

In [5]:
def predict_letter(lmlist):
    alp_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z','del','space']
    lmdf = pd.DataFrame([lmlist])
    lmdf.columns = [str(landmark)+a for landmark in mp_holistic.HandLandmark for a in ['.x','.y','.z']]
    pred = model.predict(lmdf)
    index = pred.argmax(axis=1)
    return [alp_list[index[0]],pred[0][index[0]]]


# webcam

In [ ]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #BGR to RGB (cv2mediapipe use RGB)
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        #Set flag
        image.flags.writeable = False
        
        #Detections
        results = hands.process(image)
        
        #Set flag
        image.flags.writeable = True
        
        #RGB TO BGR
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        #Detections
        #print(results)
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image,hand,mp_hands.HAND_CONNECTIONS)
            for hand in results.multi_hand_landmarks:
                lmlist =[]
                for landmark in hand.landmark:
                    lmlist.append(landmark.x)
                    lmlist.append(landmark.y)
                    lmlist.append(landmark.z)
            alp,res = predict_letter(lmlist)
            cv2.putText(image,alp+" "+str(res) ,(50, 50),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 0, 0),2)
        cv2.imshow('Hand Tracking',image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
#results.multi_hand_landmarks

In [8]:
cap.release()
cv2.destroyAllWindows()